# Applied data science capstone: Data-based decision support to inform relocations

## Introduction
Description of the problem and business case


Relocations, moving to a new place and establishing one's home there due to e.g. change of job, are periods of great changes where several important decisions need to be taken. Among these important decisions, where to live is probably one of the most important ones. In effect, one's home location determines not only how much time will be used for commuting to work/to study or how big one's home will be given an available budget, but also what kind of services (grocery shops, restaurants, schools, cinemas, etc.) will be easily accessible.

In many cases, the decision of where to relocate is taken either quickly or based on limited information, especially when one is relocating far, e.g. to another country. 

This capstone will aim at developing a data-based decision support to help those in the process of relocating. 

To simplify the decision-making process of where to relocate, it is assumed that it depends on the following parameters:

* **Composition of neighbourhood**, this is a subjective criteria that depends on the individual preferences of the person relocating. 
* **Size of the new apartment**, this is a function of available budget and the chosen location (neighbourhood) to relocate. 
* **Commuting time**, this can be modeled as a function of the distance between the chosen location to relocate and the location of the commute (work/study). 

For the purpose of this capstone, the user (i.e. the one relocating) will define its preferences and constraints in terms of:
* **Location (target neighbourhood) he would like the new apartment's location to be similar to**, this can be the current apartment's location if the user finds it is a comfortable neighbourhood. 
* **Available budget**, this will be used to estimate the size of the apartment given a recommended location
* **Location of work/study**, this will be used to estimate commuting time by computing distance between work/study location and the new apartment's location.

The main idea is that users inform i) a neighbourhood location they like, ii) the city where they are rellocating, iii) an available budget and iv) the location of work/study. 

Given the above parameters, the user will be presented with suggested neighbourhoods to relocate. For each suggested neighbourhood, an estimated apartment size and daily commuting time will be calculated. This will provide decision-support to the user, that will then be able to target their apartment search on the recommended neighbourhoods. 

This project could be extended so that not only neighbourhoods, but actually apartments, are proposed to the user relocating. 

In order to suggest neighbourhoods for relocation, the Foursquare location data will be used to characterize the 'target neighbourhood' as well as the different neighbourhoods in the city where to relocate. Then, a clustering algorithm will be used to cluster the set of the neighbourhoods in the new city plus the target neighbourhood. Once similar neighbourhoods to the target one are identified, commuting times and apartment's size will be estimated based on user-provided information. 


## Libraries
Before progressing further, let's import the necessary libraries that will be used. 

In [35]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


print('Libraries imported.')



Libraries imported.
Libraries imported.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


## Data
Where you describe the data that will be used to solve the problem and the source of the data.


### Neigbourhoods in Barcelona
We will assume that the user wants to relocate to Barcelona.

An overview of Barcelona's districts (each district contains several neighbourhoods) can be seen below: 
<img src="450px-Barcelona_districtes.svg.png" />

The coordinates of the different neighbourhoods in Barcelona will be extracted from <a href="https://en.wikipedia.org/wiki/Districts_of_Barcelona">this Wikipedia page</a>.

These coordinates will be used to explore the different neighbourhoods (plus the target one) in FOURSQUARE.



In [36]:
## Print map of Barcelona and its neighbourhoods
address = 'Barcelona, SPAIN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Barcelona are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Barcelona are 41.3828939, 2.1774322.


In [37]:
# create map of Barcelona using latitude and longitude values
map_barcelona = folium.Map(location=[latitude, longitude], zoom_start=12)
map_barcelona

### Represent Barcelona's neighbourhoods
Here we will convert into a Pandas's dataframe the coordinates of Barcelona's neighbourhoods that have been filed into an Excel file once downloaded from the above website.

In [38]:
pd.read_excel('borough_BCN.xlsx', index_col=0)
data2 = pd.read_excel('borough_BCN.xlsx', sheet_name='Sheet1')
data2.head()

,Borough,Neighborhood,Latitude,Longitude
0,Ciutat Vella,La Barceloneta,41.379889,2.189361
1,Ciutat Vella,El Gotic,41.382778,2.176944
2,Ciutat Vella,El Raval,41.379722,2.168056
3,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",41.384608,2.182717
4,Eixample,L'Antiga Esquerra de l'Eixample,41.390000,2.155000


In [39]:
data2.shape

(67, 4)

In [40]:
# create map of Barcelona using latitude and longitude values
map_barcelona = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(data2['Latitude'], data2['Longitude'], data2['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)  
    
map_barcelona

### Sqm price of Barcelona neighbourhoods

The price per square meter of an apprtment in Barcelona will be extracted from <a href="https://www.bcn.cat/estadistica/castella/dades/timm/ipreus/hab2mave/evo/t2mab.htm">https://www.bcn.cat/estadistica/castella/dades/timm/ipreus/hab2mave/evo/t2mab.htm</a>, which is provided by the local council of Barcelona. 
UARE.

## Additional user information 
For the purpose of illustrating this capstone project, the following parameters will be assumed:
* **Location (target neighbourhood) he would like the new apartment's location to be similar to**: a location in Madrid (similar city) will be chosen.
* **New work location**: the user will be working close to 'Sants Station', the main train station in Barcelona. 
* **User's available budget:** The user has an available budget of 400.000 EUR to buy the apartment where to relocate. 

### Location (target neighbourhood)
The target location is defined below:

In [41]:
## Print map of Barcelona and its neighbourhoods
address2 = 'Madrid, Barrio de Salamanca, SPAIN.'
latitude2 = 40.43
longitude2 = -3.677778
print('The target Neighbourhood is', address2, 'Its geograpical coordinates are', latitude2, ',', longitude2,'.' )

The target Neighbourhood is Madrid, Barrio de Salamanca, SPAIN. Its geograpical coordinates are 40.43 , -3.677778 .


In [42]:
# create map of Barcelona using latitude and longitude values
map_madrid = folium.Map(location=[latitude2, longitude2], zoom_start=12)

# add markers to map
folium.CircleMarker(
        [latitude2, longitude2],
        radius=5,
        popup='Barrio Salamanca',
        color='red',
        fill=True,
        fill_color='#3187cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_madrid)  
    
map_madrid

### New work location 
The new work location is defined here:

In [43]:
## Print map of Barcelona and its neighbourhoods
address3 = 'Barcelona, Sants Estacio, SPAIN.'
latitude3 = 41.380586
longitude3 = 2.140598
print('The new work location is', address3, 'Its geograpical coordinates are', latitude3, ',', longitude3,'.' )


The new work location is Barcelona, Sants Estacio, SPAIN. Its geograpical coordinates are 41.380586 , 2.140598 .


We can now show the new work location in the Map of Barcelona (cercle in red below):

In [44]:
map_barcelona = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
folium.CircleMarker(
        [latitude3, longitude3],
        radius=8,
        popup='New work location',
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)
# add markers to map
for lat, lng, label in zip(data2['Latitude'], data2['Longitude'], data2['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_barcelona)  

map_barcelona

## Methodology section 
It represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.


### Define Foursquare Credentials and Version¶
Next, we are going to utilize the Foursquare API to explore the neighborhoods and segment them.

In [45]:
CLIENT_ID = '4RERZM5X0OFLE2UCKIXI0KJKFF4Q3MM2AO02Y45BAZNARUIN' # your Foursquare ID
CLIENT_SECRET = 'M2TBIVRFVYOBXHWE2TY2YNM1T5XQCJWDRFSTEQKVA3LFVT4N' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4RERZM5X0OFLE2UCKIXI0KJKFF4Q3MM2AO02Y45BAZNARUIN
CLIENT_SECRET:M2TBIVRFVYOBXHWE2TY2YNM1T5XQCJWDRFSTEQKVA3LFVT4N


## Exploration of venues in Barcelona's neighbourhoods 

First, let's add the target neighbourhood in the list of neighbourhoods of Barcelona, so that we can explore venues of Barcelona's and target neighbourhood in one go.

In [46]:
df=pd.DataFrame([['Target Neighbourhood', address2,latitude2,longitude2]], columns=['Borough', 'Neighborhood', 'Latitude','Longitude'])
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Target Neighbourhood,"Madrid, Barrio de Salamanca, SPAIN.",40.43,-3.677778


In [47]:
result = pd.concat([data2, df])
result.reset_index(inplace = True, drop = True) 
result.head(90)

,Borough,Neighborhood,Latitude,Longitude
0,Ciutat Vella,La Barceloneta,41.379889,2.189361
1,Ciutat Vella,El Gotic,41.382778,2.176944
2,Ciutat Vella,El Raval,41.379722,2.168056
3,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",41.384608,2.182717
4,Eixample,L'Antiga Esquerra de l'Eixample,41.390000,2.155000
5,Eixample,La Nova Esquerra de l'Eixample,41.383389,2.149000
6,Eixample,Dreta de l'Eixample,41.395278,2.166667
7,Eixample,Fort Pienc,41.395675,2.183703
8,Eixample,Sagrada Família,41.403561,2.174347
9,Eixample,Sant Antoni,41.377778,2.161111


Then, let's create a function to collect venues in all the neighborhoods in Barcelona and in the target neighborhood (Madrid):

In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
LIMIT=500
barcelona_venues = getNearbyVenues(names=result['Neighborhood'],
                                   latitudes=result['Latitude'],
                                   longitudes=result['Longitude']
                                  )

La Barceloneta
El Gotic
El Raval
Sant Pere, Santa Caterina i la Ribera
L'Antiga Esquerra de l'Eixample
La Nova Esquerra de l'Eixample
Dreta de l'Eixample
Fort Pienc
Sagrada Família
Sant Antoni
La Bordeta
La Font de la Guatlla
Hostafrancs
La Marina de Port
La Marina del Prat Vermell
El Poble-sec
Sants
Sants-Badal
Montjuïc
Zona Franca – Port
Les Corts
La Maternitat i Sant Ramon
Pedralbes
El Putget i Farró
Sarrià
Sant Gervasi – la Bonanova
Sant Gervasi – Galvany
Les Tres Torres
Vallvidrera, el Tibidabo i les Planes
Vila de Gràcia
Camp d'en Grassot i Gràcia Nova
La Salut
Vallcarca i els Penitents
El Baix Guinardó
El Guinardó
Can Baró
El Carmel
La Font d'en Fargues
Horta
Montbau
La Teixonera
Vall d'Hebron
Can Peguera
Canyelles 
Ciutat Meridiana
La Guineueta
Les Roquetes
Torre Baró
La Trinitat Nova
El Turó de la Peira
Baró de Viver
Bon Pastor
El Congrés i els Indians
Navas
Sant Andreu de Palomar
La Sagrera
Trinitat Vella
El Besòs i el Maresme
El Clot
El Camp de l'Arpa del Clot
Diagonal Mar i

Let's check the size of the resulting dataframe:

In [50]:
print(barcelona_venues.shape)
barcelona_venues.head(5)

(2876, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,La Barceloneta,41.379889,2.189361,Baluard Barceloneta,41.380047,2.189250,Bakery
1,La Barceloneta,41.379889,2.189361,BRO,41.380214,2.189007,Burger Joint
2,La Barceloneta,41.379889,2.189361,La Cova Fumada,41.379254,2.189254,Tapas Restaurant
3,La Barceloneta,41.379889,2.189361,Plaça de la Barceloneta,41.379739,2.188135,Plaza
4,La Barceloneta,41.379889,2.189361,Rumbanroll,41.380597,2.187807,Mediterranean Restaurant


Let's check as well how many venues were returned for each neighbourhood.

In [51]:
barcelona_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Baró de Viver,4,4,4,4,4,4
Bon Pastor,7,7,7,7,7,7
Camp d'en Grassot i Gràcia Nova,46,46,46,46,46,46
Can Baró,20,20,20,20,20,20
Can Peguera,31,31,31,31,31,31
Canyelles,9,9,9,9,9,9
Ciutat Meridiana,8,8,8,8,8,8
Diagonal Mar i el Front Marítim del Poblenou,81,81,81,81,81,81
Dreta de l'Eixample,100,100,100,100,100,100


Let's check how many unique categories can be curated from all the returned venues:

In [52]:
print('There are {} uniques categories.'.format(len(barcelona_venues['Venue Category'].unique())))

There are 271 uniques categories.


## Exploration of venues in target neighbourhood

## 3. Analyze Each Neighborhood

In [53]:
# one hot encoding
barcelona_onehot = pd.get_dummies(barcelona_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
barcelona_onehot['Neighborhood'] = barcelona_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [barcelona_onehot.columns[-1]] + list(barcelona_onehot.columns[:-1])
barcelona_onehot = barcelona_onehot[fixed_columns]

barcelona_onehot.head()

,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Building,Burger Joint,Bus Line,Bus Station,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Country Dance Club,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pharmacy,Photography Lab,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Climbing Spot,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Transportation Service,Tunnel,Turkish Restaurant,Udon Restaurant,University,Vacation Rental,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [54]:
barcelona_onehot.shape

(2876, 271)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [55]:
barcelona_grouped = barcelona_onehot.groupby('Neighborhood').mean().reset_index()
barcelona_grouped

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Buffet,Building,Burger Joint,Bus Line,Bus Station,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,College Residence Hall,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Country Dance Club,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pharmacy,Photography Lab,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Climbing Spot,Rock Club,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Transportation Service,Tunnel,Turkish Restaurant,Udon Restaurant,University,Vacation Rental,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Women's Store
0,Baró de Viver,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.000000,

Let's check the new size of the dataframe:

In [56]:
barcelona_grouped.shape

(67, 271)

#### Let's print each neighborhood along with the top 5 most common venues

In [57]:
num_top_venues = 5

for hood in barcelona_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = barcelona_grouped[barcelona_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Baró de Viver----
          venue  freq
0  Burger Joint  0.25
1         Plaza  0.25
2          Park  0.25
3          Café  0.25
4   Yoga Studio  0.00


----Bon Pastor----
                   venue  freq
0          Metro Station  0.29
1                   Café  0.29
2             Restaurant  0.14
3  Performing Arts Venue  0.14
4      Electronics Store  0.14


----Camp d'en Grassot i Gràcia Nova----
                      venue  freq
0       Japanese Restaurant  0.09
1                    Bakery  0.09
2                     Hotel  0.07
3          Tapas Restaurant  0.07
4  Mediterranean Restaurant  0.04


----Can Baró----
                venue  freq
0  Spanish Restaurant  0.20
1       Grocery Store  0.10
2      Scenic Lookout  0.10
3               Plaza  0.10
4    Tapas Restaurant  0.05


----Can Peguera----
                venue  freq
0  Spanish Restaurant  0.10
1               Plaza  0.10
2       Grocery Store  0.10
3         Supermarket  0.10
4  Seafood Restaurant  0.06


----Canyelles 

4  Spanish Restaurant  0.05


----Montjuïc----
                venue  freq
0                Park  0.18
1              Garden  0.12
2       Historic Site  0.12
3  Athletics & Sports  0.06
4               Diner  0.06


----Navas----
                  venue  freq
0           Supermarket  0.15
1         Grocery Store  0.09
2            Restaurant  0.09
3  Gym / Fitness Center  0.06
4                  Café  0.06


----Pedralbes----
                      venue  freq
0      Gym / Fitness Center  0.18
1  Mediterranean Restaurant  0.18
2                     Hotel  0.09
3        Spanish Restaurant  0.09
4                    Garden  0.09


----Poblenou----
                      venue  freq
0  Mediterranean Restaurant  0.07
1        Spanish Restaurant  0.07
2                Restaurant  0.06
3                    Bakery  0.05
4        Italian Restaurant  0.05


----Provençals del Poblenou----
                      venue  freq
0               Pizza Place  0.09
1  Mediterranean Restaurant  0.09
2     

Let's put that into a pandas dataframe

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = barcelona_grouped['Neighborhood']

for ind in np.arange(barcelona_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(barcelona_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Baró de Viver,Park,Plaza,Burger Joint,Café,Women's Store,Fast Food Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market
1,Bon Pastor,Café,Metro Station,Electronics Store,Restaurant,Performing Arts Venue,Farmers Market,Escape Room,Exhibit,Fabric Shop,Falafel Restaurant
2,Camp d'en Grassot i Gràcia Nova,Bakery,Japanese Restaurant,Hotel,Tapas Restaurant,Diner,Bar,Spanish Restaurant,Mediterranean Restaurant,Supermarket,Restaurant
3,Can Baró,Spanish Restaurant,Grocery Store,Plaza,Scenic Lookout,Basketball Court,Dessert Shop,Tapas Restaurant,Bar,Big Box Store,Italian Restaurant
4,Can Peguera,Plaza,Supermarket,Spanish Restaurant,Grocery Store,Pizza Place,Bakery,Seafood Restaurant,Tapas Restaurant,Café,Escape Room


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 10 clusters.

In [60]:
# set number of clusters
kclusters = 20

barcelona_grouped_clustering = barcelona_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(barcelona_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([ 4, 11,  7,  2,  9,  5, 17,  7,  7,  9])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [61]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [62]:
neighborhoods_venues_sorted.head()
barcelona_merged = result.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
barcelona_merged.dropna(axis=0, inplace=True)
barcelona_merged.head(100)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ciutat Vella,La Barceloneta,41.379889,2.189361,7.0,Tapas Restaurant,Paella Restaurant,Spanish Restaurant,Mediterranean Restaurant,Seafood Restaurant,Restaurant,Bar,Burger Joint,Plaza,Beer Bar
1,Ciutat Vella,El Gotic,41.382778,2.176944,1.0,Tapas Restaurant,Plaza,Ice Cream Shop,Bar,Spanish Restaurant,Hotel,Italian Restaurant,Coffee Shop,Japanese Restaurant,Market
2,Ciutat Vella,El Raval,41.379722,2.168056,1.0,Mediterranean Restaurant,Tapas Restaurant,Cocktail Bar,Spanish Restaurant,Pizza Place,Bar,Café,Bookstore,Gastropub,Italian Restaurant
3,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",41.384608,2.182717,1.0,Tapas Restaurant,Hotel,Bar,Plaza,Cocktail Bar,Spanish Restaurant,Seafood Restaurant,Restaurant,Mediterranean Restaurant,Café
4,Eixample,L'Antiga Esquerra de l'Eixample,41.390000,2.155000,7.0,Cocktail Bar,Spanish Restaurant,Hotel,Mediterranean Restaurant,Japanese Restaurant,Sandwich Place,Bakery,Restaurant,Tapas Restaurant,Seafood Restaurant
5,Eixample,La Nova Esquerra de l'Eixample,41.383389,2.149000,0.0,Japanese Restaurant,Mediterranean Restaurant,Indian Restaurant,Café,Park,Bakery,Gym,Brewery,Supermarket,Electronics Store
6,Eixample,Dreta de l'Eixample,41.395278,2.166667,7.0,Hotel,Boutique,Mediterranean Restaurant,Hostel,Restaurant,Bakery,Tapas Restaurant,Seafood Restaurant,Café,Roof Deck
7,Eixample,Fort Pienc,41.395675,2.183703,9.0,Coffee Shop,Restaurant,Theater,Gym / Fitness Center,Sushi Restaurant,Bistro,Bakery,Juice Bar,Park,Szechuan Restaurant
8,Eixample,Sagrada Família,41.403561,2.174347,7.0,Italian Restaurant,Burger Joint,Hotel,Plaza,Bakery,Pizza Place,Restaurant,Supermarket,Japanese Restaurant,Latin American Restaurant
9,Eixample,Sant Antoni,41.377778,2.161111,1.0,Tapas Restaurant,Cocktail Bar,Mediterranean Restaurant,Spanish Restaurant,Café,Bar,Burger Joint,Pizza Place,Coffee Shop,Argentinian Restaurant


In [63]:
#barcelona_merged = result

#barcelona_merged = barcelona_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#print(barcelona_merged)

In [64]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



In [65]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(barcelona_merged['Latitude'], barcelona_merged['Longitude'], barcelona_merged['Neighborhood'], barcelona_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [66]:
barcelona_merged.loc[barcelona_merged['Cluster Labels'] == 7, barcelona_merged.columns[[1] + list(range(5, barcelona_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,La Barceloneta,Tapas Restaurant,Paella Restaurant,Spanish Restaurant,Mediterranean Restaurant,Seafood Restaurant,Restaurant,Bar,Burger Joint,Plaza,Beer Bar
4,L'Antiga Esquerra de l'Eixample,Cocktail Bar,Spanish Restaurant,Hotel,Mediterranean Restaurant,Japanese Restaurant,Sandwich Place,Bakery,Restaurant,Tapas Restaurant,Seafood Restaurant
6,Dreta de l'Eixample,Hotel,Boutique,Mediterranean Restaurant,Hostel,Restaurant,Bakery,Tapas Restaurant,Seafood Restaurant,Café,Roof Deck
8,Sagrada Família,Italian Restaurant,Burger Joint,Hotel,Plaza,Bakery,Pizza Place,Restaurant,Supermarket,Japanese Restaurant,Latin American Restaurant
11,La Font de la Guatlla,Spanish Restaurant,Nightclub,Plaza,Supermarket,Art Museum,Gay Bar,Bakery,Hotel,Café,Tapas Restaurant
12,Hostafrancs,Tapas Restaurant,Hotel,Mediterranean Restaurant,Restaurant,Spanish Restaurant,Pizza Place,Breakfast Spot,Bar,Bakery,Asian Restaurant
17,Sants-Badal,Pizza Place,Mediterranean Restaurant,Bakery,Supermarket,Hotel,Bar,Restaurant,Hostel,Sushi Restaurant,Fruit & Vegetable Store
21,La Maternitat i Sant Ramon,Spanish Restaurant,Café,Restaurant,Vegetarian / Vegan Restaurant,Soccer Stadium,Hotel,Diner,Pizza Place,Soccer Field,Football Stadium
22,Pedralbes,Gym / Fitness Center,Mediterranean Restaurant,Spanish Restaurant,Plaza,Hotel,Tennis Court,Trail,Museum,Garden,Exhibit
23,El Putget i Farró,Bakery,Food & Drink Shop,Pizza Place,Wine Bar,Tea Room,Italian Restaurant,Molecular Gastronomy Restaurant,Gastropub,Tapas Restaurant,Thai Restaurant


## Results 
From the clustering exercise, we can see that the target neighborhood (i.e. Barrio de Salamanca, Madrid) belongs to cluster 7. This cluster is characterized by a high densiy of restauration services, e.g. bars, restaurants, cafés.




## Discussion 
For the sake of illustration, we consider two neighourhoods in Barcelona (i.e. city where the end user wants to move) also clustered in cluster 7, namely: 
* Poblenou
* Sarrià

Based on Google maps and on the information mentioned above regarding housing prices in Barcelona, see below commuting time and surface of apartment (calculated based on informed budget) for the two neighbourhood candidates based on the clustering analysis: 

* Poblenou:


Commuting time: 30 minutes


Surface of apartment: 85 sqm

* Sarrià


Commuting time: 25 minutes


Surface of apartment: 73




## Conclusion 
The present capstone project illustrates how end-users planning to relocate can be offered recomendations in terms of neighbourhoods based on the following: 

* Target eighbourhood (i.e. neighbourhood they like)
* City where they want to relocate
* Available budget to buy an apartment in the new city
* Location of new place of work / study

Through application of clustering algorithms, recommended neighbourhoods are identifid based on similarities of venues obtained through FOURSQUARE. Then, user input is used to estimate commuting time and surface of apartments. 

The proposed methodology has been illustrated and two neighbourhoods have been proposed; one closer to the new place of work where sqm are high resulting in short commute but small apartment, and another a little further away from the work place but with cheaper sqm prices, leading to higher surfaces. Based on this information, the end-user can make an informed decision of where to relocate based on his preferences.